In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from data import CIFAR10
from ff import FF, FFLayer
import torch.nn as nn

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
batch_size = 32

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=True, transform=transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./datasets', train=False,
                                       download=True, transform=transform)

In [ ]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Inference

In [ ]:
threshold = 2.0
epochs = 25
model = FF()
model.add_layer(FFLayer(nn.Linear(3468, 1024), optimizer=torch.optim.Adam, epochs=epochs, threshold=threshold, activation=nn.ReLU(), lr=0.01, positive_lr=0.005, negative_lr=0.005))
model.add_layer(FFLayer(nn.Linear(1024, 512), optimizer=torch.optim.Adam, epochs=epochs, threshold=threshold, activation=nn.ReLU(), lr=0.01, positive_lr=0.005, negative_lr=0.005))
model.add_layer(FFLayer(nn.Linear(512, 256), optimizer=torch.optim.Adam, epochs=epochs, threshold=threshold, activation=nn.ReLU(), lr=0.01, positive_lr=0.005, negative_lr=0.005))

In [ ]:
model.train()

In [ ]:
x = iter(trainloader)

In [ ]:
x, y = next(x)

In [ ]:
new_x, new_y = CIFAR10.overlay_y_on_x(x, y)

In [ ]:
model(new_x)

# Training

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=True, transform=transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
model.train()
for i, (x, y) in enumerate(trainloader):
    
    x_pos, _ =  CIFAR10.overlay_y_on_x(x, y)
    rnd = torch.randperm(x.size(0))
    x_neg, _ = CIFAR10.overlay_y_on_x(x, y[rnd])
    losses = model(x_pos, x_neg)
    print(losses)

In [ ]:
predictions, real = CIFAR10.predict(testloader, model)
print("Accuracy ", np.sum(predictions == real)/len(real))